In [11]:
import yfinance as yf
import pandas as pd
def optimal_insurance_put(symbol):
    ticker=yf.Ticker(symbol)
    current_price=yf.download(symbol)['Adj Close'][-1]
    puts=ticker.option_chain().puts
    optimal_puts=puts[(puts['strike']>0.9*(current_price)) & (puts['strike']<1.1*(current_price))]
    optimal_puts['Insurance cost']=optimal_puts['strike']-optimal_puts['ask']-current_price

    purchasable_op=optimal_puts[(optimal_puts['strike']<current_price)]
    purchasable_op['Cost margin']=purchasable_op['Insurance cost']/current_price
    ideal_option=purchasable_op[purchasable_op['Insurance cost']==purchasable_op['Insurance cost'].min()]

    return ideal_option


In [12]:
optimal_insurance_put('AAPL')

[*********************100%%**********************]  1 of 1 completed
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_74498/2668414907.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimal_puts['Insurance cost']=optimal_puts['strike']-optimal_puts['ask']-current_price
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_74498/2668414907.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchasable_op['Cost margin']=purchasable_op['Insurance cost']/current_price


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,Insurance cost,Cost margin
22,AAPL241004P00207500,2024-09-27 19:59:24+00:00,207.5,0.08,0.08,0.1,-0.03,-27.272728,2038.0,4665,0.373053,False,REGULAR,USD,-20.389993,-0.089512


### Testing average cost of insurance puts for above function

In [3]:
tickers=['AMZN','BP','MSFT','TTE','AAPL','NVDA']
margin=[]
for ticker in tickers:

    margin.append(optimal_puts(ticker)['Cost margin'].tolist())

margin = [item for sublist in margin for item in sublist]

print(f'{round(sum(margin)/len(tickers)*100,2)}%')

[*********************100%%**********************]  1 of 1 completed
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_74498/518862504.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimal_puts['Insurance cost']=optimal_puts['strike']-optimal_puts['ask']-current_price
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_74498/518862504.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchasable_op['Cost margin']=purchasable_op['Insurance cost']/current_price
[*********************

## Aggresive options strategy (CALLS)

In [ ]:
def optimal_insurance_put(symbol):
    ticker=yf.Ticker(symbol)
    current_price=yf.download(symbol)['Adj Close'][-1]
    puts=ticker.option_chain().call
    optimal_puts=puts[(puts['strike']>0.9*(current_price)) & (puts['strike']<1.1*(current_price))]
    optimal_puts['Insurance cost']=optimal_puts['strike']-optimal_puts['ask']-current_price

    purchasable_op=optimal_puts[(optimal_puts['strike']<current_price)]
    purchasable_op['Cost margin']=purchasable_op['Insurance cost']/current_price
    ideal_option=purchasable_op[purchasable_op['Insurance cost']==purchasable_op['Insurance cost'].min()]

    return ideal_option

In [20]:
symbol='AAPL'
ticker=yf.Ticker(symbol)
current_price=yf.download(symbol)['Adj Close'][-1]
calls=ticker.option_chain().calls
calls['Status']=''
for i in range(len(calls)):
    if current_price>calls['strike'][i]+calls['bid'][i]:
        calls['Status'][i]=="Buy Right now"

    elif current_price>calls['strike'][i]:
        calls['Status'][i]=="Buy considering Bid price"


[*********************100%%**********************]  1 of 1 completed


In [31]:
calls.max()

contractSymbol             AAPL241004C00300000
lastTradeDate        2024-09-27 19:59:59+00:00
strike                                   300.0
lastPrice                                116.7
bid                                      127.7
ask                                      128.3
change                                6.700005
percentChange                         66.66667
volume                                 35119.0
openInterest                             15861
impliedVolatility                     2.816409
inTheMoney                                True
contractSize                           REGULAR
currency                                   USD
Status                                        
d_1                                   1.314464
d_2                                   0.787113
BSM call val                        205.032663
Diff to mkt                         191.143452
dtype: object

In [27]:
import numpy as np
from scipy.stats import norm
import math
calls['d_1']=(np.log(current_price/calls['strike'])+(0.02+(calls['impliedVolatility']/2))*5)/(calls['impliedVolatility']*np.sqrt(5))#Where t==5
calls['d_2']=calls['d_1']-calls['impliedVolatility']*np.sqrt(5)


cdf_1=norm.cdf(calls['d_1'])
cdf_2=norm.cdf(calls['d_2'])
calls['BSM call val']=cdf_1*current_price-(cdf_2*calls['strike']*math.exp(-0.02*5))



In [29]:
calls['Diff to mkt']=abs(calls['BSM call val']-current_price)

In [36]:

pd.DataFrame(calls.loc[calls['Diff to mkt'].idxmax()])

,32
contractSymbol,AAPL241004C00240000
lastTradeDate,2024-09-27 19:56:25+00:00
strike,240.0
lastPrice,0.08
bid,0.08
ask,0.09
change,-0.05
percentChange,-38.461536
volume,5575.0
openInterest,9867
